In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sn

In [ ]:
data = pd.read_csv('carprice.csv')
data

In [ ]:
data.info()

In [ ]:
data.isna().sum()

In [ ]:
data=data.drop(['car_ID'], axis =1)


In [ ]:
data['CarName'].head()

In [ ]:
data['Company'] = data['CarName'].apply(lambda x:x.split(' ')[0]) 
data['Company'].head()

In [ ]:
data = data.drop(['CarName'], axis =1)

In [ ]:
data['Company'].value_counts()

In [ ]:
data['Company'].replace('toyouta', 'toyota',inplace=True)
data['Company'].replace('Nissan', 'nissan',inplace=True)
data['Company'].replace('maxda', 'mazda',inplace=True)
data['Company'].replace('vokswagen', 'volkswagen',inplace=True)
data['Company'].replace('vw', 'volkswagen',inplace=True)
data['Company'].replace('porcshce', 'porsche',inplace=True)

data['Company'].value_counts()

In [ ]:
objcols = data.select_dtypes(include='object')
objcols

In [ ]:
numbers = { 'one': 1, 'two': 2, 'three': 3, 'four': 4, 'five': 5, 'six': 6, 'seven': 7, 'eight': 8, 'nine': 9, 'ten': 10, 'eleven':11, 'twelve': 12 }

data = data.replace(numbers)
data

In [ ]:

independent_variables = data.drop(['price'], axis=1).columns

for column in independent_variables:
    plt.figure(figsize=(5, 5))

    if data[column].dtype == 'object':
        sn.boxplot(x=column, y='price', data=data)
        plt.xticks(rotation=90)
        plt.title(f'{column} vs. Price')

    else:
        plt.bar(data[column], data['price'], data=data )
        plt.xlabel(column)
        plt.ylabel('Price')
        plt.title(f'{column} vs. Price')
    
    plt.grid(True)
    plt.show()

In [ ]:
data=data.drop(['doornumber','carheight','carwidth','enginetype','fuelsystem'], axis =1)


In [ ]:
objcols = data.select_dtypes(include='object')
objcols

In [ ]:
data['symboling'].unique()

In [ ]:
data = data.replace({
        'fueltype': {'gas': 0, 'diesel': 1},
        'aspiration': {'std': 0, 'turbo': 1},
        'carbody': {'convertible': 0, 'hatchback': 1, 'sedan': 2, 'wagon': 3, 'hardtop': 4},
        'drivewheel': {'rwd': 0, 'fwd': 1, '4wd': 2},
        'enginelocation': {'front': 0, 'rear': 1},
        'symboling': {-3: 0, -2: 1, -1: 2, 0: 3, 1: 4, 2: 5, 3: 6}
    })



In [ ]:
compnames = {}
ls = data['Company'].unique()
for i in range(22):
    compnames[ls[i]] = i
data = data.replace(compnames)


In [ ]:
objcols = data.select_dtypes(include='object')
objcols

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

X = data.drop('price', axis=1)  
y = data['price']            

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
from sklearn.model_selection import cross_val_score

estimators = [100, 200, 300, 400, 500]

cv_scorelist = []

# Performing k-fold cross-validation for each number of estimators

for n in estimators:
    
    random_forest = RandomForestRegressor(n_estimators=n, random_state=42)
    
    # Perform 5-fold cross-validation and get the R2 scores for each fold
    scores = cross_val_score(random_forest, X, y, cv=5, scoring='r2')
    
    # Calculate the mean R2 score across all folds
    mean_r2_scores = np.mean(scores)
    
    cv_scorelist.append(mean_r2_scores)

best_index = np.argmax(cv_scorelist)

best_n_estimators = estimators[best_index]

In [ ]:
random_forest = RandomForestRegressor(n_estimators=best_n_estimators, random_state=42)
random_forest.fit(X_train, y_train)

y_pred = random_forest.predict(X_test)

r2 = r2_score(y_test, y_pred)

print("R-squared (R2):", r2*100)

In [ ]:
comparison_df = pd.DataFrame({'Real Price': y_test, 'Expected Price': y_pred})
print(comparison_df)

In [ ]:
plt.figure(figsize=(8, 6))
plt.scatter(y_pred, y_test, color='blue', label='Actual vs. Predicted')
plt.plot([min(y_pred), max(y_pred)], [min(y_test), max(y_test)], color='red', linestyle='-', label='Ideal Line')
plt.xlabel('Predicted Values (y_pred)')
plt.ylabel('Actual Values (y_test)')
plt.title('Actual vs. Predicted Values')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
user = {
    'symboling': 4,'fueltype': 'gas','aspiration':'std','carbody':'sedan','drivewheel':'fwd','enginelocation':'front','wheelbase':105.8,'carlength':192.7,
    'curbweight':2844,'cylindernumber': 'five','enginesize': 136,'boreratio': 3.19,'stroke':3.4,'compressionratio':8.5,'horsepower' : 110,
    'peakrpm': 5500,'citympg':19,'highwaympg':25,'Company': 'audi'
}

    
userdata = pd.DataFrame([user])

userdata = userdata.replace({
        'fueltype': {'gas': 0, 'diesel': 1},
        'aspiration': {'std': 0, 'turbo': 1},
        'carbody': {'convertible': 0, 'hatchback': 1, 'sedan': 2, 'wagon': 3, 'hardtop': 4},
        'drivewheel': {'rwd': 0, 'fwd': 1, '4wd': 2},
        'enginelocation': {'front': 0, 'rear': 1},
        'symboling': {-3: 0, -2: 1, -1: 2, 0: 3, 1: 4, 2: 5, 3: 6}
    })


userdata = userdata.replace(compnames)
userdata = userdata.replace(numbers)

predicted_price = random_forest.predict(userdata)

print("Predicted Price:", predicted_price[0])